# LOG

In [ ]:
import glob
import os
from lxml import etree
import pandas as pd

In [ ]:
# Load the files to log together

folder_annexA = ''
folder_CINCensus = ''

In [ ]:
# Import CIN Census files

files = os.listdir(dir_input_cin)

In [ ]:
NS = {None:'http://www.dcsf.gov.uk/schemas/cbds'}

In [ ]:
files = 'C:/Users/celine.gross/Social Finance Ltd/CS Front Door Data Collaboration - Documents/06. Sensitive data (restricted)/West Berkshire/input/cin/CIN Final Extract 31032018.xml'
tree = etree.parse(files)
root = tree.getroot()

In [ ]:
children = root.findall('.//Child', NS)
for child in children:
    print(child)

In [ ]:
child = children[0]

In [ ]:
def findlist(element, tag, NS):
    '''
    Starting from the 'element', makes a list of the contents of 'tag' nieces (siblings' children sharing the same tag)
    '''
    value_list = []
    values = element.getparent().findall('.//{}'.format(tag), NS)
    for value in values:
        value_list.append(value.text.strip())
    return value_list

In [ ]:
def get_ChildIdentifiers(element, NS=None):
    childidentifiers = {}
    for group in element:
        column = etree.QName(group).localname
        value = group.text
        childidentifiers[column] = value
    return childidentifiers

In [ ]:
def get_ChildCharacteristics(element, NS):
    childcharacteristics = {}
    for group in element:
        if group.tag.endswith('Ethnicity'):
            column = etree.QName(group).localname
            value = group.text
        elif group.tag.endswith('Disabilities'):
            column = etree.QName(group).localname
            value = findlist(group, 'Disability', NS)
        childcharacteristics[column] = value
    return childcharacteristics

In [ ]:
def get_Group(element, NS):
    group = {}
    # Load our reference element
    group['Date'] = element.text
    group['Type'] = etree.QName(element).localname
    # Get the other elements on the same level (siblings)
    siblings = element.getparent().getchildren()
    for sibling in siblings:
        if sibling is not element:
            if len(sibling.getchildren())==0: # if siblings don't have children, just get their value
                column = etree.QName(sibling).localname
                value = sibling.text
                group[column] = [value]
    # If we're in the Assessment or ChildProtectionPlans modules, we need to get down one level
    # to collect all AssessmentFactors abd CPPreviewDate
    if element.getparent().tag.endswith('Assessments'):
        group['Factors'] = findlist(element, 'AssessmentFactors', NS)
    if element.getparent().tag.endswith('ChildProtectionPlans'):
        group['CPPreview'] = findlist(element, 'CPPreviewDate', NS)
    return group

In [ ]:
for child in children:
    for item in child:
        if item.tag.endswith('ChildIdentifiers'):
            childid = get_ChildIdentifiers(item)
        elif item.tag.endswith('CINdetails'):
            for i in item:
                if i.tag.endswith('Assessments'):
                    assts = i.findall('AssessmentActualStartDate', NS)
                    for a in assts:
                        assts_info = get_Group(a, NS)
                    print(assts_info)

In [ ]:
for child in children:
    

In [ ]:
child = children[1]
event_list = ['CINreferralDate', 'CINclosureDate']
all_events_data = pd.DataFrame()

for group in child:
    if group.tag.endswith('ChildIdentifiers'):
        childidentifiers = get_ChildIdentifiers(group)
    if group.tag.endswith('ChildCharacteristics'):
        childcharacteristics = get_ChildCharacteristics(group, NS)
    
    if group.tag.endswith('CINdetails'):
#        all_events_data = pd.DataFrame()
        for event_type in event_list:
            event = group.findall(event_type, NS)
            for e in event:
                cinref = get_Group(e, NS)
                df = pd.DataFrame(cinref)
                all_events_data = pd.concat([all_events_data, df])
                print(type(all_events_data))

for key, value in childcharacteristics.items() :
    print(value)
    all_events_data[key] = [value]
for key, value in list(childidentifiers.items()) :
    all_events_data[key] = value
print(all_events_data)

In [ ]:
test1 = pd.DataFrame({'coucou': 'Chris', 'ghgh': 'Celine'}, index=[1])
test2 = pd.DataFrame({'salut': 'Chris', 'bonjour': 'Celine'}, index=[1])
test3 = pd.concat([test1, test2], axis =0)
test3

In [ ]:
child.getchildren()[2].getchildren()